In [1]:
using CondensedMatterSOS
@spin σ[1:2]

((CondensedMatterSOS.SpinVariable[σˣ₁, σˣ₂], CondensedMatterSOS.SpinVariable[σʸ₁, σʸ₂], CondensedMatterSOS.SpinVariable[σᶻ₁, σᶻ₂]),)

In [2]:
function hamiltonian(σ)
    σx, σy, σz = σ
    N = length(σx)
    return -sum(σx[n] * σx[n+1] for n in 1:(N-1)) - σx[N] * σx[1] - sum(σz)
end
H = 1.0 * hamiltonian(σ)

(-2.0 + 0.0im)σˣ₁σˣ₂ + -σᶻ₁ + -σᶻ₂

In [3]:
import MultivariatePolynomials
const MP = MultivariatePolynomials
v = MP.monovec([1, σx[1], σx[2], σy[1], σy[2], σz[1], σz[2]])

7-element Array{CondensedMatterSOS.SpinMonomial,1}:
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂
 1

min_σ H(σ) = -2√2

max γ s.t. H >= γ -> H - γ = v' Q v

In [4]:
monos = CondensedMatterSOS.MP.monovec(v)

7-element Array{CondensedMatterSOS.SpinMonomial,1}:
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂
 1

In [5]:
function _mat(x::Vector)
    Q = Matrix{eltype(x)}(undef, n, n)
    k = 0
    for j in 1:n
        for i in 1:j
            k += 1
            Q[i, j] = Q[j, i] = x[k]
        end
    end
    return Q
end
import MathOptInterface
const MOI = MathOptInterface
function _mapcoef(op, f::MOI.ScalarAffineFunction)
    MOI.ScalarAffineFunction([
        MOI.ScalarAffineTerm(op(t.coefficient), t.variable_index) for t in f.terms
    ], op(f.constant))
end

_mapcoef (generic function with 1 method)

In [6]:
using MosekTools
optimizer = Mosek.Optimizer()
n = length(v)
x, cx = MOI.add_constrained_variables(optimizer, MOI.PositiveSemidefiniteConeTriangle(n))
γ = MOI.SingleVariable(MOI.add_variable(optimizer))
F = MOI.ScalarAffineFunction{Complex{Float64}}
Q = _mat(F.(MOI.SingleVariable.(x)))
gram = sum(Q[i, j] * (1.0 * monos[i] * monos[j]) for i in 1:n for j in 1:n)
z = H - convert(F, γ) - gram
for c in MP.coefficients(z)
    for a in [_mapcoef(real, c), _mapcoef(imag, c)]
        MOI.Utilities.normalize_and_add_constraint(optimizer, a, MOI.EqualTo(0.0))
    end
end
MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MAX_SENSE)
MOI.set(optimizer, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
MOI.optimize!(optimizer)

MOSEK warning 500: The license expires in 3 days.
MOSEK warning 500: The license expires in 3 days.
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 32              
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :           

1-element Array{MosekTools.MosekSolution,1}:
 MosekTools.MosekSolution(Mosek.MSK_SOL_ITR, Mosek.MSK_SOL_STA_OPTIMAL, Mosek.MSK_PRO_STA_PRIM_AND_DUAL_FEAS, Mosek.Stakey[Mosek.MSK_SK_SUPBAS], [-3.999999999796037], [[1.000000000057635, -9.269581886647066e-24, -5.656210524136898e-31, -0.9999999998611814, 3.3633595397551523e-35, -4.980160841047063e-33, -3.582423614962809e-32, 2.79402353463965e-10, 1.0222415923756377e-20, 9.697247170385857e-24  …  1.0000000000576352, -2.3368415203743696e-26, 5.290417929564324e-31, -3.525638415242312e-31, 2.7940235346396994e-10, 2.5699158727710933e-20, -7.523378745640498e-21, 0.49999914277708546, -0.4999999999305953, 1.0000017145395945]], [-0.0], [-0.0], [-0.0], Mosek.Stakey[Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX  …  Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX, Mosek.MSK_SK_FIX

In [7]:
@show MOI.get(optimizer, MOI.TerminationStatus())

MOI.get(optimizer, MOI.TerminationStatus()) = MathOptInterface.OPTIMAL


OPTIMAL::TerminationStatusCode = 1

In [8]:
MOI.get(optimizer, MOI.ObjectiveValue())

-3.999999999796037

In [9]:
round.(_mat(MOI.get(optimizer, MOI.ConstraintDual(), cx)), digits=6)

7×7 Array{Float64,2}:
  1.0  -0.0  -0.0   1.0  -0.0  -0.0  -0.0
 -0.0   1.0   0.0  -0.0   0.0   0.0   0.0
 -0.0   0.0   1.0  -0.0  -0.0   1.0   1.0
  1.0  -0.0  -0.0   1.0  -0.0  -0.0  -0.0
 -0.0   0.0  -0.0  -0.0   1.0  -0.0  -0.0
 -0.0   0.0   1.0  -0.0  -0.0   1.0   1.0
 -0.0   0.0   1.0  -0.0  -0.0   1.0   1.0

In [10]:
using LinearAlgebra
Γ = Hermitian([
    1 0 0   0     0     1/2 1/2
    0 1 1/2 1/2im 0     0   0
    0 0 1   0     1/2im 0   0
    0 0 0   1    -1/2   0   0
    0 0 0   0     1     0   0
    0 0 0   0     0     1   1
    0 0 0   0     0     0   1
])

7×7 Hermitian{Complex{Float64},Array{Complex{Float64},2}}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im  …   0.0+0.0im  0.5+0.0im  0.5+0.0im
 0.0-0.0im  1.0+0.0im  0.5+0.0im      0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0-0.0im  0.5-0.0im  1.0+0.0im      0.0-0.5im  0.0+0.0im  0.0+0.0im
 0.0-0.0im  0.0+0.5im  0.0-0.0im     -0.5+0.0im  0.0+0.0im  0.0+0.0im
 0.0-0.0im  0.0-0.0im  0.0+0.5im      1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.5-0.0im  0.0-0.0im  0.0-0.0im  …   0.0-0.0im  1.0+0.0im  1.0+0.0im
 0.5-0.0im  0.0-0.0im  0.0-0.0im      0.0-0.0im  1.0-0.0im  1.0+0.0im